# Analytic center computation

# The set-up

In [123]:
import numpy as np
import pandas as pd
import accpm
import accpm
from IPython.display import display

$\DeclareMathOperator{\domain}{dom}
\newcommand{\transpose}{\text{T}}
\newcommand{\vec}[1]{\begin{pmatrix}#1\end{pmatrix}}$

# Summary of results

From the testing undertaken here we can conclude that parameter values $\texttt{alpha = 0.01}$ and $\texttt{beta = 0.99}$ yield acceptable results for squares not too large or too small.

# Theory

To test the $\texttt{analytic_center}$ function we consider the following example. Suppose we want to find the analytic center $x_{ac} \in \mathbb{R}^2$ of the inequalities $x_1 \leq c_1, x_1 \geq 0, x_2 \leq c_2, x_2 \geq 0$. This is a rectange with dimensions $c_1 \times c_2$ centered at at $(\frac{c_1}{2}, \frac{c_2}{2})$ so we should have $x_{ac} = (\frac{c_1}{2}, \frac{c_2}{2})$. Now, $x_{ac}$ is the solution of the minimization problem 
\begin{equation*}
    \min_{\domain \phi} \phi(x) = - \sum_{i=1}^{4}{\log{(b_i - a_i^\transpose x)}}
\end{equation*}
where 
\begin{equation*}
    \domain \phi = \{x \;|\; a_i^\transpose x < b_i, i = 1, 2, 3, 4\}
\end{equation*}
with
\begin{align*}
    &a_1 = \begin{bmatrix}1\\0\end{bmatrix}, &&b_1 = c_1, \\
    &a_2 = \begin{bmatrix}-1\\0\end{bmatrix}, &&b_2 = 0, \\
    &a_3 = \begin{bmatrix}0\\1\end{bmatrix}, &&b_3 = c_2, \\
    &a_4 = \begin{bmatrix}0\\-1\end{bmatrix}, &&b_4 = 0. 
\end{align*}
So we solve
\begin{align*}
    &\phantom{iff}\nabla \phi(x) = \sum_{i=1}^{4
    } \frac{1}{b_i - a_i^\transpose x}a_i = 0 \\
    &\iff \frac{1}{c_1-x_1}\begin{bmatrix}1\\0\end{bmatrix} + \frac{1}{x_1}\begin{bmatrix}-1\\0\end{bmatrix} + \frac{1}{c_2-x_2}\begin{bmatrix}0\\1\end{bmatrix} + \frac{1}{x_2}\begin{bmatrix}0\\-1\end{bmatrix} = 0 \\
    &\iff \frac{1}{c_1-x_1} - \frac{1}{x_1} = 0, \frac{1}{c_2-x_2} - \frac{1}{x_2} = 0 \\
    &\iff x_1 = \frac{c_1}{2}, x_2 = \frac{c_2}{2},
\end{align*}
as expected. 

# Testing

We test $\texttt{analytic_center}$ for varying values of $c_1, c_2$ and algorithm parameters $\texttt{alpha, beta}$:

In [124]:
def get_results(A, test_input, alpha, beta, tol=10e-8):
    expected = []
    actual = []
    result = []
    for (c1, c2) in test_input:
        b = np.array([c1, 0, c2, 0])
        ac_expected = np.asarray((c1/2, c2/2))
        ac_actual = accpm.analytic_center(A, b, alpha = alpha, beta = beta)
        expected.append(ac_expected)
        actual.append(ac_actual)
        # if np.array_equal(ac_expected, ac_actual):
        if np.linalg.norm(ac_expected - ac_actual) <= tol: 
            result.append(True)
        else:
            result.append(False)
    results = pd.DataFrame([test_input, expected, actual, result])
    results = results.transpose()
    results.columns = ['test_input', 'expected', 'actual', 'result']
    print('alpha =', alpha, 'beta =', beta)
    display(results) 

Here we have results for squares of varying sizes with $\texttt{alpha = 0.01}$ and $\texttt{beta = 0.99}$.

In [125]:
alpha = 0.01
beta = 0.99
A = np.array([[1, 0],[-1,0],[0,1],[0,-1]])
test_input = [(1, 1), (5, 5), (20, 20), (10e2, 10e2), (10e4, 10e4),
              (10e6, 10e6), (10e8, 10e8), (10e10, 10e10),  
              (0.5, 0.5), (0.1, 0.1), (0.01, 0.01), 
              (0.008, 0.008), (0.006, 0.006), (0.004, 0.004), 
              (0.003, 0.003), (0.002, 0.002), (0.001, 0.001),
              (0.0008, 0.0008), (0.0007, 0.0007), (0.0006, 0.0006),
              (0.0005, 0.0005), (0.0004, 0.0004), (0.0003, 0.0003)] 

We see that the algorithm performs quite well with these parameter values given that the size of the square is not too large, above a size of roughly $10^7 \times 10^7$, or too small, below a size of roughly $0.0004 \times 0.0004$.

In [126]:
get_results(A, test_input, alpha, beta)

alpha = 0.01 beta = 0.99


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[485730187.136, 485730187.136]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2072171021.4, 2072171021.4]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


Here we have results for squares of varying sizes with $\texttt{alpha = 0.01}$ and $\texttt{beta = 0.7}$ and then varying values for $\texttt{alpha}$ in the interval $(0, 0.5)$.

In [127]:
alpha = 0.01
beta = 0.7

For $\texttt{alpha = 0.01}$ and $\texttt{beta = 0.7}$ the algorithm performs slightly worse on larger squares but significantly worse on smaller squares, with results for squares smaller than $0.01 \times 0.01$ being unacceptable. Moreover, adjusting $\texttt{alpha}$ to be smaller or larger does not seem to yield much improvement/

In [128]:
get_results(A, test_input, alpha, beta)

alpha = 0.01 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601754358.0, 2601754358.0]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [129]:
alpha = 0.0001

In [130]:
get_results(A, test_input, alpha, beta)

alpha = 0.0001 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601754358.0, 2601754358.0]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [131]:
alpha = 0.1

In [132]:
get_results(A, test_input, alpha, beta)

alpha = 0.1 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601754358.0, 2601754358.0]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [133]:
alpha = 0.25

In [134]:
get_results(A, test_input, alpha, beta)

alpha = 0.25 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601754358.0, 2601754358.0]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [135]:
alpha = 0.4

In [136]:
get_results(A, test_input, alpha, beta)

alpha = 0.4 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601285971.72, 2601285971.72]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [137]:
alpha = 0.45

In [138]:
get_results(A, test_input, alpha, beta)

alpha = 0.45 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601285971.72, 2601285971.72]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True


In [139]:
alpha = 0.49

In [140]:
get_results(A, test_input, alpha, beta)

alpha = 0.49 beta = 0.7


,test_input,expected,actual,result
0,"(1, 1)","[0.5, 0.5]","[0.5, 0.5]",True
1,"(5, 5)","[2.5, 2.5]","[2.5, 2.5]",True
2,"(20, 20)","[10.0, 10.0]","[10.0, 10.0]",True
3,"(1000.0, 1000.0)","[500.0, 500.0]","[499.999999999, 499.999999999]",True
4,"(100000.0, 100000.0)","[50000.0, 50000.0]","[50000.0, 50000.0]",True
5,"(10000000.0, 10000000.0)","[5000000.0, 5000000.0]","[4999999.99063, 4999999.99063]",False
6,"(1000000000.0, 1000000000.0)","[500000000.0, 500000000.0]","[466684322.534, 466684322.534]",False
7,"(100000000000.0, 100000000000.0)","[50000000000.0, 50000000000.0]","[2601285971.72, 2601285971.72]",False
8,"(0.5, 0.5)","[0.25, 0.25]","[0.25, 0.25]",True
9,"(0.1, 0.1)","[0.05, 0.05]","[0.05, 0.05]",True
